In [1]:
import tensorflow as tf
keras = tf.keras
import os
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
!pip install pandas

     |████████████████████████████████| 10.1 MB 3.5 MB/s eta 0:00:01    |███████████████████▍            | 6.1 MB 2.2 MB/s eta 0:00:02█████████████████████▋         | 7.1 MB 3.5 MB/s eta 0:00:01     |███████████████████████████████▌| 9.9 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 13.7 MB/s eta 0:00:01


In [9]:
!pip install sklearn

     |████████████████████████████████| 7.1 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 6.4 MB/s eta 0:00:01     |███▍                            | 30 kB 25.4 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=661bfc0e4e73301b04d979d537f0cb677ad85d6ca96af6182e4d7c6c4d7c0471
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [24]:
import pandas as pd

In [25]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Load Data

In [26]:
df = pd.read_csv('beijing_master.csv')
df_gr = df.groupby('date').mean()
df_gr = df_gr.drop(['year', 'hour'], axis = 1)

In [27]:
train = df_gr.iloc[:-365]
test = df_gr.iloc[-365:]

In [28]:
feats = ['month', 'day', 'weekday','SO2', 'NO2', 'CO', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM','E', 'ENE', 'ESE', 'N', 'NE', 'NNE',
       'NNW', 'NW', 'S', 'SE', 'SSE', 'SSW', 'SW', 'W', 'WNW', 'WSW', 'AQI']

LSTM: Univariate 2 Univariate

In [29]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dropout

In [35]:
from lstm_supervised_conv import series_to_supervised

# Baseline Avg

In [31]:
baseline_avg = [140.62]*len(df_gr['AQI'].iloc[-400:-365])

baseline_ms = mean_squared_error(baseline_avg, df_gr['AQI'].iloc[-400:-365])
rms_baseline = np.sqrt(baseline_ms)

# Prediction 15 days ahead

In [32]:
t_lag = [1, 3, 5, 7, 10, 15, 20, 30] 
pred_15 = {}

In [272]:
#2 lstms, 1 hidden 
rms_15 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
#     model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:15])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:15]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:15].reshape(15,1), test_X[:15]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_15.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_15[(2,2)] = rms_15

Epoch 1/100
28/28 [==============================] - 4s 141ms/step - loss: 0.0357 - val_loss: 0.0226
Epoch 2/100
28/28 [==============================] - 1s 40ms/step - loss: 0.0193 - val_loss: 0.0207
Epoch 3/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0167 - val_loss: 0.0191
Epoch 4/100
28/28 [==============================] - 1s 25ms/step - loss: 0.0161 - val_loss: 0.0189
Epoch 5/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 6/100
28/28 [==============================] - 1s 24ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 7/100
28/28 [==============================] - 1s 26ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 8/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 9/100
28/28 [==============================] - ETA: 0s - loss: 0.016 - 1s 24ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 10/100
28/28 [==============================] - 1s 24ms/step - loss: 

Epoch 82/100
28/28 [==============================] - 1s 34ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 83/100
28/28 [==============================] - 1s 38ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 84/100
28/28 [==============================] - 1s 29ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 85/100
28/28 [==============================] - 1s 24ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 86/100
28/28 [==============================] - 1s 31ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 87/100
28/28 [==============================] - 1s 29ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 88/100
28/28 [==============================] - 1s 26ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 89/100
28/28 [==============================] - 1s 30ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 90/100
28/28 [==============================] - 1s 24ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 91/100
28/28 [==============================] - 1s 48ms/step - loss: 0.0159 - val_los

Epoch 44/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 45/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 46/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 47/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 48/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 49/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 50/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 51/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 52/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 53/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_los

28/28 [==============================] - 0s 9ms/step - loss: 0.0153 - val_loss: 0.0191
Epoch 7/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0151 - val_loss: 0.0190
Epoch 8/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0151 - val_loss: 0.0189
Epoch 9/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0189
Epoch 10/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0150 - val_loss: 0.0189
Epoch 11/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 12/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 13/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 14/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 15/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 16/1

28/28 [==============================] - 0s 9ms/step - loss: 0.0139 - val_loss: 0.0204
Epoch 69/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0139 - val_loss: 0.0205
Epoch 70/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0138 - val_loss: 0.0205
Epoch 71/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0138 - val_loss: 0.0205
Epoch 72/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0205
Epoch 73/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0138 - val_loss: 0.0206
Epoch 74/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0138 - val_loss: 0.0206
Epoch 75/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0138 - val_loss: 0.0206
Epoch 76/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0206
Epoch 77/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0138 - val_loss: 0.0207
Epoch 78/

27/27 [==============================] - 0s 9ms/step - loss: 0.0146 - val_loss: 0.0196
Epoch 31/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0145 - val_loss: 0.0197
Epoch 32/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0145 - val_loss: 0.0197
Epoch 33/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0145 - val_loss: 0.0197
Epoch 34/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0144 - val_loss: 0.0198
Epoch 35/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0144 - val_loss: 0.0198
Epoch 36/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0144 - val_loss: 0.0199
Epoch 37/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0143 - val_loss: 0.0199
Epoch 38/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0200
Epoch 39/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0143 - val_loss: 0.0200
Epoch 

In [274]:
#single LSTM layer, single hidden
rms_15 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
#     model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
#     model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:15])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:15]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:15].reshape(15,1), test_X[:15]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_15.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_15[(1,2)] = rms_15



Epoch 1/100
28/28 [==============================] - 1s 38ms/step - loss: 0.0336 - val_loss: 0.0218
Epoch 2/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0171 - val_loss: 0.0195
Epoch 3/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0162 - val_loss: 0.0189
Epoch 4/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 5/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 6/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 8/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0161 - val_loss: 0.0189
Epoch 9/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0160 - val_loss: 0.0189
Epoch 10/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0189
Epoc

Epoch 83/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 84/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 85/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 86/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 87/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 88/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 89/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 90/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 91/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 92/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0159 - val_loss: 0.018

Epoch 65/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 66/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 67/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 68/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 69/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 70/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 71/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0184
Epoch 72/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0151 - val_loss: 0.0184
Epoch 73/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0151 - val_loss: 0.0184
Epoch 74/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0151 - val_loss: 0.01

Epoch 47/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 48/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 49/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 50/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 51/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 52/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 53/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 54/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 55/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 56/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0185

Epoch 29/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 30/100
28/28 [==============================] - 0s 16ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 31/100
28/28 [==============================] - 1s 20ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 32/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 33/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 34/100
28/28 [==============================] - 1s 33ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 35/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 36/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 37/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 38/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.

28/28 [==============================] - 1s 31ms/step - loss: 0.0142 - val_loss: 0.0194
Epoch 92/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0142 - val_loss: 0.0194
Epoch 93/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.0194
Epoch 94/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0194
Epoch 95/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0142 - val_loss: 0.0194
Epoch 96/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 97/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 98/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 99/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 100/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0195
Epoc

28/28 [==============================] - 1s 35ms/step - loss: 0.0139 - val_loss: 0.0202
Epoch 71/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0139 - val_loss: 0.0202
Epoch 72/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0138 - val_loss: 0.0202
Epoch 73/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0202
Epoch 74/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 75/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 76/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 77/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 78/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 79/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 80/1

27/27 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0189
Epoch 53/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 54/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 55/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 56/100
27/27 [==============================] - 0s 6ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 57/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 58/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 59/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0139 - val_loss: 0.0190
Epoch 60/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0139 - val_loss: 0.0191
Epoch 61/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0139 - val_loss: 0.0191
Epoch 62/100


27/27 [==============================] - 0s 6ms/step - loss: 0.0144 - val_loss: 0.0195
Epoch 35/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0143 - val_loss: 0.0195
Epoch 36/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0143 - val_loss: 0.0195
Epoch 37/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 38/100
27/27 [==============================] - 0s 15ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 39/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0141 - val_loss: 0.0196
Epoch 40/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0196
Epoch 41/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0140 - val_loss: 0.0196
Epoch 42/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0140 - val_loss: 0.0196
Epoch 43/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0139 - val_loss: 0.0197
Epoch 44

In [275]:
#single LSTM layer, single hidden
rms_15 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
#     model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
#     model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:15])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:15]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:15].reshape(15,1), test_X[:15]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_15.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_15[(1,2)] = rms_15




Epoch 1/100
28/28 [==============================] - 1s 21ms/step - loss: 0.0335 - val_loss: 0.0216
Epoch 2/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0171 - val_loss: 0.0195
Epoch 3/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 4/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 5/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 6/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 7/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 8/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0189
Epoch 9/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0189
Epoch 10/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0189
Epoch 11

28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 65/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 66/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 67/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 68/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 69/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 70/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 71/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 72/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 73/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 74/100


28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 47/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 48/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 49/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 50/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 51/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 52/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 53/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 54/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 55/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 56/100

Epoch 28/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 29/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 30/100
28/28 [==============================] - 0s 17ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 31/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 32/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 33/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 34/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 35/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0149 - val_loss: 0.0186
Epoch 36/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0149 - val_loss: 0.0186
Epoch 37/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0149 - val_loss: 0.

Epoch 10/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 11/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 12/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 13/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 14/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 15/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 16/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 17/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 18/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0148 - val_loss: 0.0188
Epoch 19/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0148 - val_loss: 0.018

Epoch 92/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 93/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 94/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 95/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 96/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 97/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 98/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 99/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 100/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 1/100
28/28 [==============================] - 1s 22ms/step - loss: 0.0271 - val_loss: 0.0

Epoch 74/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 75/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 76/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 77/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 78/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 79/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 80/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0204
Epoch 81/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0137 - val_loss: 0.0205
Epoch 82/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0205
Epoch 83/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0137 - val_loss: 0.0205


Epoch 56/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0189
Epoch 57/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 58/100
27/27 [==============================] - 0s 17ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 59/100
27/27 [==============================] - 0s 18ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 60/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 61/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 62/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 63/100
27/27 [==============================] - 0s 6ms/step - loss: 0.0139 - val_loss: 0.0191
Epoch 64/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0139 - val_loss: 0.0191
Epoch 65/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0139 - val_loss: 0.01

Epoch 38/100
27/27 [==============================] - 0s 15ms/step - loss: 0.0143 - val_loss: 0.0199
Epoch 39/100
27/27 [==============================] - 0s 16ms/step - loss: 0.0142 - val_loss: 0.0199
Epoch 40/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.0199
Epoch 41/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0199
Epoch 42/100
27/27 [==============================] - 0s 6ms/step - loss: 0.0141 - val_loss: 0.0199
Epoch 43/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0140 - val_loss: 0.0199
Epoch 44/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0139 - val_loss: 0.0200
Epoch 45/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0139 - val_loss: 0.0200
Epoch 46/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0200
Epoch 47/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.020

In [273]:
#2 lstm 2 hidden

t_lag = [1, 3, 5, 7, 10, 15, 20, 30] 
rms_15 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
    model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:15])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:15]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:15].reshape(15,1), test_X[:15]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_15.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_15[(2,3)] = rms_15

Epoch 1/100
28/28 [==============================] - 1s 39ms/step - loss: 0.0311 - val_loss: 0.0259
Epoch 2/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0201 - val_loss: 0.0216
Epoch 3/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0172 - val_loss: 0.0195
Epoch 4/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0161 - val_loss: 0.0187
Epoch 5/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 6/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 7/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 8/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 9/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 10/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0160 - val_loss: 0.0186
Epoch 

28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 64/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 65/100
28/28 [==============================] - 1s 22ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 66/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 67/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 68/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 69/100
28/28 [==============================] - 1s 18ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 70/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 71/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 72/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch

28/28 [==============================] - 0s 12ms/step - loss: 0.0151 - val_loss: 0.0186
Epoch 26/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0186
Epoch 27/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0186
Epoch 28/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0186
Epoch 29/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0151 - val_loss: 0.0186
Epoch 30/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 31/100
28/28 [==============================] - 0s 16ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 32/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 33/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 34/100
28/28 [==============================] - 0s 17ms/step - loss: 0.0150 - val_loss: 0.0186
Epo

Epoch 87/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 88/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 89/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 90/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 91/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 92/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 93/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 94/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 95/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 96/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss

27/27 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0191
Epoch 50/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0192
Epoch 51/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0192
Epoch 52/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0192
Epoch 53/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0193
Epoch 54/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0141 - val_loss: 0.0193
Epoch 55/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0140 - val_loss: 0.0193
Epoch 56/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0140 - val_loss: 0.0193
Epoch 57/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0140 - val_loss: 0.0194
Epoch 58/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0140 - val_loss: 0.0194
Epoch 5

# Predict 30 days ahead

In [ ]:
pred_30 = {}

In [276]:
#2 LSTM 1 hidden
t_lag = [1, 3, 5, 7, 10, 15, 20, 30] 
rms_30 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
#     model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:30])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:30]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:30].reshape(30,1), test_X[:30]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_30.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_30[(2,2)] = rms_30

Epoch 1/100
28/28 [==============================] - 1s 51ms/step - loss: 0.0362 - val_loss: 0.0228
Epoch 2/100
28/28 [==============================] - 1s 26ms/step - loss: 0.0195 - val_loss: 0.0209
Epoch 3/100
28/28 [==============================] - 1s 21ms/step - loss: 0.0168 - val_loss: 0.0191
Epoch 4/100
28/28 [==============================] - 1s 19ms/step - loss: 0.0161 - val_loss: 0.0189
Epoch 5/100
28/28 [==============================] - 1s 19ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 6/100
28/28 [==============================] - 1s 25ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 7/100
28/28 [==============================] - 0s 17ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 8/100
28/28 [==============================] - 0s 17ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 9/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 10/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0160 - val_loss: 0.0187

28/28 [==============================] - 1s 22ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 64/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 65/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 66/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 67/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 68/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 69/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 70/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 71/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 72/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 73

28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 26/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 27/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 28/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 29/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 30/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 31/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 32/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 33/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 34/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0149 - val_loss: 0.0186
Epoch 35/10

28/28 [==============================] - 0s 12ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 88/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 89/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 90/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 91/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 92/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 93/100
28/28 [==============================] - 1s 25ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 94/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 95/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch 96/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0142 - val_loss: 0.0198
Epoch

27/27 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 50/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0191
Epoch 51/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0191
Epoch 52/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 53/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 54/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 55/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 56/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 57/100
27/27 [==============================] - 0s 18ms/step - loss: 0.0140 - val_loss: 0.0191
Epoch 58/100
27/27 [==============================] - 1s 21ms/step - loss: 0.0140 - val_loss: 0.0192
Epo

In [277]:
#1 LSTM 1 hidden
rms_30 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
#     model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
#     model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:30])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:30]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:30].reshape(30,1), test_X[:30]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_30.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_30[(1,2)] = rms_30

Epoch 1/100
28/28 [==============================] - 1s 19ms/step - loss: 0.0342 - val_loss: 0.0225
Epoch 2/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0171 - val_loss: 0.0196
Epoch 3/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0162 - val_loss: 0.0189
Epoch 4/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0187
Epoch 5/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 6/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 7/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 8/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0161 - val_loss: 0.0188
Epoch 9/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 10/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 11

Epoch 83/100
28/28 [==============================] - 1s 19ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 84/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 85/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 86/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 87/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 88/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 89/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 90/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 91/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 92/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.01

Epoch 65/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 66/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 67/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 68/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 69/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 70/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 71/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 72/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 73/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 74/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0152 - val_loss: 0.0

Epoch 47/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 48/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 49/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 50/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 51/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 52/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 53/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0186
Epoch 54/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0186
Epoch 55/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0186
Epoch 56/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0186


Epoch 29/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 30/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 31/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 32/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 33/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 34/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 35/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 36/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 37/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0185
Epoch 38/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.01

Epoch 11/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 12/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 13/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 14/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 15/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 16/100
28/28 [==============================] - ETA: 0s - loss: 0.0137- ETA: 0s - loss: 0.01 - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 17/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0148 - val_loss: 0.0188
Epoch 18/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0148 - val_loss: 0.0188
Epoch 19/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0148 - val_loss: 0.0188
Epoch 20/100
28/28 [==============================] -

28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 93/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 94/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 95/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 96/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 97/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 98/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 99/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 100/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 1/100
28/28 [==============================] - 1s 19ms/step - loss: 0.0279 - val_loss: 0.0255
Epoch 2/100


28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 75/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 76/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0138 - val_loss: 0.0204
Epoch 77/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0204
Epoch 78/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0204
Epoch 79/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0204
Epoch 80/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0205
Epoch 81/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0205
Epoch 82/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0205
Epoch 83/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0205
Epoch 84/100


27/27 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 57/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 58/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 59/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 60/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 61/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0190
Epoch 62/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0140 - val_loss: 0.0190
Epoch 63/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0140 - val_loss: 0.0191
Epoch 64/100
27/27 [==============================] - 0s 16ms/step - loss: 0.0140 - val_loss: 0.0191
Epoch 65/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0140 - val_loss: 0.0191
Epoch 66/1

Epoch 38/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0145 - val_loss: 0.0201
Epoch 39/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0144 - val_loss: 0.0201
Epoch 40/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0144 - val_loss: 0.0201
Epoch 41/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0144 - val_loss: 0.0201
Epoch 42/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0143 - val_loss: 0.0201
Epoch 43/100
27/27 [==============================] - 0s 12ms/step - loss: 0.0143 - val_loss: 0.0201
Epoch 44/100
27/27 [==============================] - 0s 13ms/step - loss: 0.0142 - val_loss: 0.0201
Epoch 45/100
27/27 [==============================] - 0s 15ms/step - loss: 0.0142 - val_loss: 0.0202
Epoch 46/100
27/27 [==============================] - 0s 6ms/step - loss: 0.0141 - val_loss: 0.0202
Epoch 47/100
27/27 [==============================] - 0s 7ms/step - loss: 0.0141 - val_loss: 0.0

In [37]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [38]:
#1 LSTM 2 hidden
rms_30 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
#     model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
    model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:30])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:30]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:30].reshape(30,1), test_X[:30]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_30.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_30[(1,3)] = rms_30

Epoch 1/100
28/28 [==============================] - 1s 23ms/step - loss: 0.0267 - val_loss: 0.0219
Epoch 2/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0177 - val_loss: 0.0194
Epoch 3/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0164 - val_loss: 0.0190
Epoch 4/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0161 - val_loss: 0.0189
Epoch 5/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 6/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 7/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0188
Epoch 8/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 9/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 10/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0187
Epoch 11

Epoch 83/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 84/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 85/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 86/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 87/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 88/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 89/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 90/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 91/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0159 - val_loss: 0.0186
Epoch 92/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0158 - val_loss: 0.0

28/28 [==============================] - 0s 8ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 65/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 66/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 67/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 68/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 69/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 70/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 71/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 72/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 73/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 74/100


28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 47/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 48/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 49/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 50/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 51/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 52/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 53/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 54/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 55/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0186
Epoch 56/100


28/28 [==============================] - 0s 7ms/step - loss: 0.0151 - val_loss: 0.0185
Epoch 29/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 30/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 31/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 32/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 33/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 34/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 35/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 36/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 37/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0185
Epoch 38/100


28/28 [==============================] - 0s 7ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 11/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 12/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 13/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 14/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 15/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 16/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 17/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 18/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 19/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 20/100


Epoch 92/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0143 - val_loss: 0.0195
Epoch 93/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0143 - val_loss: 0.0195
Epoch 94/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0143 - val_loss: 0.0195
Epoch 95/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0143 - val_loss: 0.0195
Epoch 96/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 97/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 98/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 99/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0195
Epoch 100/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0142 - val_loss: 0.0196
Epoch 1/100
28/28 [==============================] - 1s 21ms/step - loss: 0.0290 - val_loss: 0.0255

28/28 [==============================] - 0s 12ms/step - loss: 0.0137 - val_loss: 0.0201
Epoch 74/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0137 - val_loss: 0.0202
Epoch 75/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0137 - val_loss: 0.0202
Epoch 76/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0202
Epoch 77/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0203
Epoch 78/100
28/28 [==============================] - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0203
Epoch 79/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0137 - val_loss: 0.0203
Epoch 80/100
28/28 [==============================] - 0s 8ms/step - loss: 0.0137 - val_loss: 0.0203
Epoch 81/100
28/28 [==============================] - ETA: 0s - loss: 0.013 - 0s 7ms/step - loss: 0.0137 - val_loss: 0.0204
Epoch 82/100
28/28 [==============================] - 0s 6ms/step - loss: 0.0136 - val_

KeyboardInterrupt: 

In [43]:
#1 LSTM 2 hidden
rms_30 = []
p_30 = {}

batch_sizes = [5, 10, 20, 30, 40, 50]

for b in batch_sizes: 

    reframed = series_to_supervised(df_gr['AQI'], 3, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    #     model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
    model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')

    history = model.fit(train_X, train_y, epochs= b, batch_size=42,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:30])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:30]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:30].reshape(30,1), test_X[:30]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    p_30[b] = np.sqrt(mean_squared_error(inv_yhat, inv_y))

p_30

Epoch 1/5
27/27 [==============================] - 1s 21ms/step - loss: 0.0271 - val_loss: 0.0245
Epoch 2/5
27/27 [==============================] - 0s 7ms/step - loss: 0.0177 - val_loss: 0.0215
Epoch 3/5
27/27 [==============================] - 0s 6ms/step - loss: 0.0163 - val_loss: 0.0205
Epoch 4/5
27/27 [==============================] - 0s 6ms/step - loss: 0.0157 - val_loss: 0.0196
Epoch 5/5
27/27 [==============================] - 0s 6ms/step - loss: 0.0154 - val_loss: 0.0193
Epoch 1/10
27/27 [==============================] - 1s 20ms/step - loss: 0.0304 - val_loss: 0.0248
Epoch 2/10
27/27 [==============================] - 0s 7ms/step - loss: 0.0191 - val_loss: 0.0211
Epoch 3/10
27/27 [==============================] - 0s 6ms/step - loss: 0.0166 - val_loss: 0.0205
Epoch 4/10
27/27 [==============================] - 0s 6ms/step - loss: 0.0159 - val_loss: 0.0200
Epoch 5/10
27/27 [==============================] - 0s 6ms/step - loss: 0.0155 - val_loss: 0.0195
Epoch 6/10
27/27 [=====

Epoch 19/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0190
Epoch 20/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0190
Epoch 21/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0190
Epoch 22/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0190
Epoch 23/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0152 - val_loss: 0.0190
Epoch 24/40
27/27 [==============================] - 0s 7ms/step - loss: 0.0151 - val_loss: 0.0190
Epoch 25/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0151 - val_loss: 0.0190
Epoch 26/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0151 - val_loss: 0.0190
Epoch 27/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0151 - val_loss: 0.0190
Epoch 28/40
27/27 [==============================] - 0s 6ms/step - loss: 0.0151 - val_loss: 0.0190
Epoch 29/4

{5: 66.8892842840959,
 10: 66.84858410808619,
 20: 66.69975549844926,
 30: 66.74503113810488,
 40: 66.68999657886191,
 50: 66.80995522848973}

In [279]:
#2 LSTM 2 hidden
rms_30 = []
for t in t_lag:
    reframed = series_to_supervised(df_gr['AQI'], t, 1)
    values = reframed.values
    train = values[:-365, :]
    test = values[-365:, :]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.fit_transform(test)

    train_X, train_y = scaled_train[:, :-1], scaled_train[:, -1]
    test_X, test_y = scaled_test[:, :-1], scaled_test[:, -1]

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences = True))
    model.add(Dense(50, activation = 'tanh'))
    model.add(Dense(25, activation ='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    
    history = model.fit(train_X, train_y, epochs= 100, batch_size=40,  verbose=1, shuffle=False
                                   , validation_data = (test_X, test_y))

    yhat = model.predict(test_X[:30])
    yhat = yhat.reshape(yhat.shape[0], yhat.shape[1])
    test_X = test_X.reshape(test_X.shape[0], test_X.shape[2])
    inv_yhat = np.concatenate((yhat, test_X[:30]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:, 0]


    inv_y = np.concatenate((test_y[:30].reshape(30,1), test_X[:30]), axis=1)
    inv_y = scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rms_30.append(np.sqrt(mean_squared_error(inv_yhat, inv_y)))

pred_30[(2,3)] = rms_30

Epoch 1/100
28/28 [==============================] - 1s 34ms/step - loss: 0.0312 - val_loss: 0.0260
Epoch 2/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0202 - val_loss: 0.0216
Epoch 3/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0172 - val_loss: 0.0195
Epoch 4/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0161 - val_loss: 0.0187
Epoch 5/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 6/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 7/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 8/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 9/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0160 - val_loss: 0.0187
Epoch 10/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0160 - val_loss: 0.0186


28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0184
Epoch 64/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 65/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 66/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 67/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 68/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 69/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 70/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 71/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0152 - val_loss: 0.0185
Epoch 72/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0152 - val_loss: 0.0185
Epoc

Epoch 45/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0187
Epoch 46/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0187
Epoch 47/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0187
Epoch 48/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0151 - val_loss: 0.0187
Epoch 49/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0151 - val_loss: 0.0187
Epoch 50/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0151 - val_loss: 0.0187
Epoch 51/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 52/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 53/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0150 - val_loss: 0.0187
Epoch 54/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0150 - val_loss

28/28 [==============================] - 0s 13ms/step - loss: 0.0151 - val_loss: 0.0188
Epoch 8/100
28/28 [==============================] - 1s 20ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 9/100
28/28 [==============================] - 1s 27ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 10/100
28/28 [==============================] - 0s 16ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 11/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 12/100
28/28 [==============================] - 1s 24ms/step - loss: 0.0150 - val_loss: 0.0188
Epoch 13/100
28/28 [==============================] - 1s 18ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 14/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 15/100
28/28 [==============================] - 0s 16ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch 16/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0149 - val_loss: 0.0188
Epoch

28/28 [==============================] - 0s 12ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 89/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 90/100
28/28 [==============================] - 1s 33ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 91/100
28/28 [==============================] - 1s 25ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 92/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 93/100
28/28 [==============================] - 0s 10ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 94/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 95/100
28/28 [==============================] - 0s 11ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 96/100
28/28 [==============================] - 0s 12ms/step - loss: 0.0143 - val_loss: 0.0198
Epoch 97/100
28/28 [==============================] - 0s 15ms/step - loss: 0.0143 - val_loss: 0.0199
Epo

27/27 [==============================] - 0s 15ms/step - loss: 0.0143 - val_loss: 0.0190
Epoch 51/100
27/27 [==============================] - 1s 20ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 52/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 53/100
27/27 [==============================] - 0s 11ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 54/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 55/100
27/27 [==============================] - 0s 8ms/step - loss: 0.0142 - val_loss: 0.0190
Epoch 56/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 57/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 58/100
27/27 [==============================] - 0s 9ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 59/100
27/27 [==============================] - 0s 10ms/step - loss: 0.0141 - val_loss: 0.0191
Epoch 6

In [7]:
#view rms_15 and rms_30 for best network config (#lstm layers, #hidden)